In [1]:
import numpy as np
import pandas as pd

In [2]:
timesteps = 14
startDay = 350

In [3]:
dt = pd.read_csv("m5-forecasting-accuracy/sales_train_evaluation.csv")
dt.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1


In [4]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int64(1941), object(6)
memory usage: 452.9+ MB


In [5]:
#To reduce memory usage
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [6]:
#Reduce memory usage and compare with the previous one to be sure
dt = downcast_dtypes(dt)

In [7]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


In [8]:
#Take the transpose so that we have one day for each row, and 30490 items' sales as columns
dt = dt.T    
dt.head(8)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
item_id,HOBBIES_1_001,HOBBIES_1_002,HOBBIES_1_003,HOBBIES_1_004,HOBBIES_1_005,HOBBIES_1_006,HOBBIES_1_007,HOBBIES_1_008,HOBBIES_1_009,HOBBIES_1_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
dept_id,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,...,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3
cat_id,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,...,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS
store_id,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,...,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3
state_id,CA,CA,CA,CA,CA,CA,CA,CA,CA,CA,...,WI,WI,WI,WI,WI,WI,WI,WI,WI,WI
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0


In [9]:
#Remove id, item_id, dept_id, cat_id, store_id, state_id columns
dt = dt[6 + startDay:]
dt.head(5)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0


In [10]:
calendar = pd.read_csv("m5-forecasting-accuracy/calendar.csv")
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [11]:
#Create dataframe with zeros for 1969 days in the calendar
daysBeforeEvent = pd.DataFrame(np.zeros((1969,1)))

In [12]:
# "1" is assigned to the days before the event_name_1. Since "event_name_2" is rare, it was not added.
for x,y in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][x])) == False):
           daysBeforeEvent[0][x-1] = 1 
            #if first day was an event this row will cause an exception because "x-1".
            #Since it is not i did not consider for now.

In [13]:
#"calendar" won't be used anymore. 
del calendar

In [14]:
#"daysBeforeEventTest" will be used as input for predicting (We will forecast the days 1913-1941)
daysBeforeEventTest = daysBeforeEvent[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
daysBeforeEvent = daysBeforeEvent[startDay:1941]

In [15]:
#Before concatanation with our main data "dt", indexes are made same and column name is changed to "oneDayBeforeEvent"
daysBeforeEvent.columns = ["oneDayBeforeEvent"]
daysBeforeEvent.index = dt.index

In [16]:
dt = pd.concat([dt, daysBeforeEvent], axis = 1)

dt.columns

Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
                     30481,               30482,               30483,
                     30484,               30485,               30486,
                     30487,               30488,               30489,
       'oneDayBeforeEvent'],
      dtype='object', length=30491)

In [17]:
dt

,0,1,2,3,4,5,6,7,8,9,...,30481,30482,30483,30484,30485,30486,30487,30488,30489,oneDayBeforeEvent
d_351,0,0,0,2,0,0,0,24,3,2,...,9,1,0,11,0,0,1,0,0,0.0
d_352,0,0,0,0,0,0,0,9,0,2,...,5,4,0,8,0,1,2,0,0,1.0
d_353,0,0,0,4,2,0,0,2,1,1,...,15,2,0,3,0,1,2,0,0,0.0
d_354,0,1,0,2,0,0,0,7,1,0,...,5,1,0,3,0,0,0,0,0,0.0
d_355,0,0,0,1,2,0,0,0,0,0,...,7,1,0,1,0,1,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1937,0,0,0,1,0,0,1,5,0,1,...,6,3,0,0,1,0,1,0,0,0.0
d_1938,3,0,2,3,0,0,0,4,0,1,...,4,3,1,2,0,1,0,1,2,0.0
d_1939,3,0,3,0,2,5,1,1,0,0,...,4,3,1,1,0,0,1,1,2,0.0
d_1940,0,0,0,2,1,2,1,40,1,0,...,1,0,0,3,1,1,0,1,5,0.0


In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming dt is a pandas DataFrame
dt.columns = dt.columns.astype(str)  # Convert column names to strings

sc = MinMaxScaler(feature_range=(0, 1))
dt_scaled = sc.fit_transform(dt)

In [19]:
X_train = []
y_train = []
for i in range(timesteps, 1941 - startDay):
    X_train.append(dt_scaled[i-timesteps:i])
    y_train.append(dt_scaled[i][0:30490]) 

In [20]:
del dt_scaled

In [21]:
#Convert to np array to be able to feed the LSTM model
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1577, 14, 30491)
(1577, 30490)


In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

# Initialize the Sequential model
dnn = Sequential()

# First layer that flattens the input
dnn.add(Flatten(input_shape=(14, 30491)))  # Adjust the input shape to match your dataset

# Adding Dense layers with Dropout for regularization
dnn.add(Dense(units=128, activation='relu'))
dnn.add(Dropout(0.2))

dnn.add(Dense(units=64, activation='relu'))
dnn.add(Dropout(0.2))

# Adjusted Output layer for multi-label classification
dnn.add(Dense(units=30490, activation='sigmoid'))  # For multi-label classification

# Compiling the model with binary crossentropy loss and adam optimizer
dnn.compile(optimizer='adam',
              loss='binary_crossentropy',  # Suitable for multi-label classification
              metrics=['accuracy'])

# Model summary to see the architecture and parameters
dnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 426874)            0         
                                                                 
 dense_3 (Dense)             (None, 128)               54640000  
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 30490)             1981850   
                                                                 
Total params: 56630106 (216.03 MB)
Trainable params: 5

In [24]:
# Fit the model to the training data, ensuring X_train and y_train are prepared correctly
history = dnn.fit(X_train, y_train,
                    epochs=32,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

Epoch 1/32
40/40 [==============================] - 4s 90ms/step - loss: 0.2962 - accuracy: 0.0000e+00 - val_loss: 0.3045 - val_accuracy: 0.0000e+00
Epoch 2/32
40/40 [==============================] - 3s 77ms/step - loss: 0.2408 - accuracy: 0.0000e+00 - val_loss: 0.3048 - val_accuracy: 0.0000e+00
Epoch 3/32
40/40 [==============================] - 3s 75ms/step - loss: 0.2372 - accuracy: 0.0000e+00 - val_loss: 0.3047 - val_accuracy: 0.0000e+00
Epoch 4/32
40/40 [==============================] - 3s 69ms/step - loss: 0.2335 - accuracy: 0.0000e+00 - val_loss: 0.3042 - val_accuracy: 0.0000e+00
Epoch 5/32
40/40 [==============================] - 3s 74ms/step - loss: 0.2324 - accuracy: 0.0000e+00 - val_loss: 0.3007 - val_accuracy: 0.0000e+00
Epoch 6/32
40/40 [==============================] - 3s 74ms/step - loss: 0.2322 - accuracy: 0.0000e+00 - val_loss: 0.3014 - val_accuracy: 0.0000e+00
Epoch 7/32
40/40 [==============================] - 3s 71ms/step - loss: 0.2299 - accuracy: 0.0000e+00 - v

In [25]:
inputs= dt[-timesteps:]
inputs = sc.transform(inputs)

In [26]:
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []

for j in range(timesteps,timesteps + 28):
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = dnn.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30491))
    testInput = np.column_stack((np.array(predicted_stock_price), daysBeforeEventTest[0][1941 + j - timesteps]))
    X_test = np.append(X_test, testInput).reshape(1,j + 1,30491)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_stock_price)

1/1 [==============================] - 0s 12ms/step


In [27]:
import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))

submission = submission.T
    
submission = pd.concat((submission, submission), ignore_index=True)

sample_submission = pd.read_csv("m5-forecasting-accuracy/sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.809802,0.752338,0.739235,0.734771,0.767322,0.766246,0.732648,0.709573,0.670736,...,0.643316,0.652321,0.663401,0.670203,0.669541,0.663912,0.656845,0.650829,0.647442,0.646068
1,HOBBIES_1_002_CA_1_validation,0.394395,0.352490,0.340872,0.336575,0.352166,0.346549,0.325798,0.311969,0.285811,...,0.244389,0.248346,0.254547,0.258454,0.258033,0.254151,0.248783,0.244275,0.241734,0.240726
2,HOBBIES_1_003_CA_1_validation,0.550464,0.445997,0.405852,0.396744,0.453995,0.548678,0.537307,0.485234,0.424717,...,0.426420,0.445807,0.457244,0.460484,0.456619,0.449582,0.443313,0.440558,0.439994,0.439286
3,HOBBIES_1_004_CA_1_validation,3.143881,2.871224,2.776696,2.748765,2.905299,3.087383,3.024823,2.890091,2.718127,...,2.729178,2.788456,2.826251,2.839067,2.828545,2.806745,2.786498,2.776889,2.774603,2.772666
4,HOBBIES_1_005_CA_1_validation,1.294279,1.195119,1.165379,1.164687,1.229482,1.236404,1.190555,1.158789,1.094606,...,1.061144,1.078501,1.099175,1.112413,1.111957,1.102162,1.089475,1.078589,1.072455,1.070063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.801000,0.710409,0.685214,0.683668,0.748838,0.804995,0.782382,0.745463,0.690316,...,0.731463,0.752078,0.769143,0.778263,0.776842,0.769314,0.761091,0.755653,0.753306,0.752318
60976,FOODS_3_824_WI_3_evaluation,0.270987,0.213422,0.198098,0.190604,0.207967,0.213560,0.191234,0.169624,0.141223,...,0.108574,0.113134,0.117754,0.119860,0.118728,0.115395,0.111497,0.108767,0.107439,0.106847
60977,FOODS_3_825_WI_3_evaluation,1.321540,1.235810,1.204822,1.201154,1.248481,1.265243,1.234117,1.203816,1.147563,...,1.102548,1.118067,1.135556,1.144310,1.141522,1.131839,1.121213,1.113987,1.110824,1.109702
60978,FOODS_3_826_WI_3_evaluation,0.281026,0.228413,0.212584,0.207488,0.222181,0.215080,0.193430,0.176547,0.149855,...,0.106589,0.109605,0.113872,0.116114,0.115207,0.112000,0.108044,0.105018,0.103441,0.102816


In [28]:
submission.to_csv("submission_dnn.csv", index=False)